In [39]:
import os
import time

import pandas as pd

import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm

In [41]:
def sel_ele_clicker(css_selector):
    sel_ele = remote.find_element(by = By.CSS_SELECTOR, 
                                  value = css_selector)
    sel_ele.click()

def bs_tbl_parser(x):
    df_sltr_bind = pd.DataFrame()
    for n_row in range(len(x)):
        x_td = x[n_row].select("td")
        val_sltr_id = x_td[1].text
        val_sltr_nm = x_td[2].text
        val_sltr_org = x_td[3].text
        val_sltr_tel = x_td[4].text
        val_sltr_addr = x_td[6].select_one("span")["onclick"].split("'")[1].strip()
        val_sltr_loc1 = x_td[9].text
        val_sltr_loc2 = x_td[10].text
        df_sltr_sub = pd.DataFrame([[val_sltr_id, val_sltr_nm, val_sltr_org,
                                     val_sltr_tel, val_sltr_addr,
                                     val_sltr_loc1, val_sltr_loc2]],
                                   columns = ["id", "name", "org", "tel", "addr", "loc1", "loc2"])
        df_sltr_bind = pd.concat([df_sltr_bind, df_sltr_sub])
    
    df_sltr_bind = df_sltr_bind.reset_index(drop = True)
    return df_sltr_bind

In [43]:
url = "https://sansatai.forest.go.kr"
val_loc = "47" # 경북지역
val_time_sleep = 2
val_page_collect = 0

In [45]:
remote = webdriver.Chrome()
remote.get(url)

In [47]:
ls_css_selector = ["li > a.mn12",
                   "#nav > div > ul > li:nth-child(7) > div > div > ul > li > a",
                   "#selectLi",
                   "#selectSido",
                   f"option[value='{val_loc}00000000']",
                   "button[title='검색']"]

In [49]:
for n_selector in range(len(ls_css_selector)):
    sel_ele_clicker(ls_css_selector[n_selector])
    time.sleep(val_time_sleep)

In [50]:
df_sltr_page_bind = pd.DataFrame()
for n_page in range(val_page_collect):
    res_source = remote.page_source
    bs_source = bs(res_source)
    bs_tbl_row = bs_source.select("#datalist tr[id]")
    df_sltr_page = bs_tbl_parser(bs_tbl_row)
    df_sltr_page_bind = pd.concat([df_sltr_page_bind, df_sltr_page])

    sel_ele_clicker("#next_pager")
    time.sleep(val_time_sleep)

In [37]:
df_sltr_page_bind.to_csv("산사태_대피소_경북_2248EA.csv", index = False,
                         encoding = "CP949")

In [53]:
remote.quit()